In [25]:
import numpy as np
import cirq
import IPython.display as IPy

### Heisenberg XX model

The Heisenberg model represents a system of $N$ particles on a line interacting with their nearest neighbours via spin coupling $J_n$, as well as with an external magnetic field $h_n$. With free boundary conditions, it has a Hamiltonian of the kind:

$$
H=\frac{1}{2}\sum\limits_{n=0}^{N-2}\left(J^x_n\sigma^x_n\sigma^x_{n+1}+J^y_n\sigma^y_n\sigma^y_{n+1}+J^z_n\sigma^z_n\sigma^z_{n+1} \right)+
\frac{1}{2}\sum\limits_{n=0}^{N-1} h_n\left(1-\sigma^z_n\right)
$$

Where $\sigma^\mu_n $ ($\mu=x,y,z$) are Pauli operators acting on each particle. This notebook covers the Heisenberg XX model, in which $J^x_n=J^y_n=J_n$ and $J^z_n=0$. In other words:

$$
H=\frac{1}{2}\sum\limits_{n=0}^{N-2}J_n\left(\sigma^x_n\sigma^x_{n+1}+\sigma^y_n\sigma^y_{n+1}\right)+
\frac{1}{2}\sum\limits_{n=0}^{N-1} h_n\left(1-\sigma^z_n\right)
$$

In [26]:
def Hamiltonian(N, J, h):
    q = cirq.LineQubit.range(N)
    x_string = [cirq.PauliString(J[n], cirq.X(q[n])*cirq.X(q[n+1])) for n in range(N-1)]
    y_string = [cirq.PauliString(J[n], cirq.Y(q[n])*cirq.Y(q[n+1])) for n in range(N-1)]
    z_string = [cirq.PauliString(h[n], cirq.Z(q[n])) for n in range(N)]
    spinspin = .5 * (cirq.PauliSum.from_pauli_strings(x_string) + cirq.PauliSum.from_pauli_strings(y_string))
    magfield = .5 * (np.sum(h) - cirq.PauliSum.from_pauli_strings(z_string))
    return spinspin + magfield

# Example/test
N = 2
print(Hamiltonian(N, np.ones(N-1), np.ones(N)))

0.500*X(q(0))*X(q(1))+0.500*Y(q(0))*Y(q(1))-0.500*Z(q(0))-0.500*Z(q(1))+1.000*I


### Jordan-Wigner Transformation

If we take the ladder operators of particle $n$, $S^{\pm}_n = (\sigma^x_n\pm i\sigma^y_n)/2 $, we may see that $\{S^+_n, S^-_n\}=I $, $\{S^+_n, S^+_n\}=\{S^-_n, S^-_n\}=0 $, which is reminiscent of a fermion's canonical anticommutation relations $\{a^\dagger_n, a_n\}=I $, $\{a^\dagger_n, a^\dagger_n\}=\{a_n, a_n\}=0$ (with $a^\dagger$, $a$ creation and annihilation operators). Motivated by this fact, we may try to translate our system of interacting particles to a system of free fermions. Notice also that the degrees of freedom remain the same: at each site we have spin up/down or a particle/vacuum.

Since ladder operators at different sites commute whereas fermionic operators should anticommute, it turns out the transformation is not so trivial [1]:

$$
\begin{aligned}
a_n &= \left(\prod\limits_{k=0}^{n-1}\sigma^z_k\right)S^+_n, &
a^\dagger_n &= \left(\prod\limits_{k=0}^{n-1}\sigma^z_k\right)S^-_n
\end{aligned}
$$

In [27]:
def a(n):
    q = cirq.LineQubit.range(n+1)
    Z = cirq.DensePauliString('Z'*n).on(*q[:n])
    Sp = .5 * (cirq.X(q[n]) + 1j * cirq.Y(q[n]))
    return Z * Sp

def ad(n):
    q = cirq.LineQubit.range(n+1)
    Z = cirq.DensePauliString('Z'*n).on(*q[:n])
    Sm = .5 * (cirq.X(q[n]) - 1j * cirq.Y(q[n]))
    return Z * Sm

# Example/test
print(a(2))

0.500*Z(q(0))*Z(q(1))*X(q(2))+0.500j*Z(q(0))*Z(q(1))*Y(q(2))


However it is easy to prove that it works. Without loss of generality take $n>m$:

$$
\{a_n, a_m\}=\left(\prod\limits_{k=0}^{n-1}\sigma^z_k\right)S^+_n\left(\prod\limits_{k=0}^{m-1}\sigma^z_k\right)S^+_m+
\left(\prod\limits_{k=0}^{m-1}\sigma^z_k\right)S^+_m\left(\prod\limits_{k=0}^{n-1}\sigma^z_k\right)S^+_n
=
$$

$$
=S^+_n\left(\prod\limits_{k=0}^{m-1}\sigma^z_k\right)\left\{S^+_m,\left(\prod\limits_{k=0}^{n-1}\sigma^z_k\right)\right\}=
S^+_n\left(\prod\limits_{k=0}^{m-1}\sigma^z_k\right)\left(\prod\limits_{k\neq m}^{n-1}\sigma^z_k\right)\{S^+_m,\sigma^z_m\}=0
$$

Notice that this calculation is essentially the same if $m>n$ or if we take the adjoint of either operator, so we have proven:

$$
\begin{aligned}
\{a^\dagger_n, a_m\}&= \delta_{nm}, & \{a_n, a_m\}&= 0
\end{aligned}
$$

In [28]:
# Test that the operators satisfy the canonical anticommutation relations
N = 5
CAR1, CAR2 = np.empty((N, N), dtype='str'), np.empty((N, N), dtype='str')

for n in range(N):
    for m in range(N):
        CAR1[n, m] = ad(n)*a(m) + a(m)*ad(n)
        CAR2[n, m] = a(n)*a(m) + a(m)*a(n)

IPy.display(IPy.Math(r'\{a^\dagger_n, a_m\}='))
print(CAR1)
IPy.display(IPy.Math(r'\{a_n, a_m\}='))
print(CAR2)

<IPython.core.display.Math object>

[['1' '0' '0' '0' '0']
 ['0' '1' '0' '0' '0']
 ['0' '0' '1' '0' '0']
 ['0' '0' '0' '1' '0']
 ['0' '0' '0' '0' '1']]


<IPython.core.display.Math object>

[['0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0']]


Inverting the transformation to obtain $S^{\pm}_n$ is trivial, and to obtain $\sigma^z_n$ we can calculate

$$
N_n=a^\dagger_n a_n =\left(\prod\limits_{k=0}^{n-1}\sigma^z_k\right)S^-_n\left(\prod\limits_{k=0}^{n-1}\sigma^z_k\right)S^+_n=S^-_nS^+_n=\frac{1}{2}\left(1-\sigma^z_n\right)
$$

So finally the fermionic Hamiltonian reads:

$$
H=\sum\limits_{n=0}^{N-2}J_n\left(a^\dagger_n a_{n+1} + a^\dagger_{n+1}a_n\right)+
\sum\limits_{n=0}^{N-1} h_n a^\dagger_n a_n
$$

An interesting observation [2] is that the total number of fermions $\mathcal{N}=\sum\limits_{n=0}^{N-1}a^\dagger_n a_n$ commutes with the Hamiltonian, therefore the number of fermions is a conserved quantity. Fermionic states are excitations $a^\dagger_{n_0}...a^\dagger_{n_k}|0\rangle$ of the vacuum state $|0\rangle$, which corresponds to the state with all spins up ($|0\rangle^{\otimes N}$) given the above definition of fermionic operators.

### Diagonalisation

What do we gain by recasting the Hamiltonian in terms of $a^\dagger_n$, $a_n$? By working in the subspace of single-particle states $\mathcal{H}^1$, with basis $|n\rangle=a^\dagger_n|0\rangle$, we find the matrix $H^1$ of the Hamiltonian is tridiagonal [2]:

$$
H^1_{nm}=\langle n|H|m \rangle = J_n\delta_{n+1,m}+J_{n-1}\delta_{n-1,m}+h_n\delta_{nm}
$$

$$
(H^1)= 
\begin{pmatrix}
h_0    & J_0    & 0      & 0       & \cdots  & 0       \\
J_0    & h_1    & J_1    & 0       & \cdots  & 0       \\
0      & J_1    & h_2    & J_2     & \cdots  & 0       \\
0      & 0      & J_2    & \ddots  & \ddots  & \vdots  \\
\vdots & \vdots & \vdots & \ddots  & \ddots  & J_{N-2} \\
0      & 0      & 0      & \cdots  & J_{N-2} & h_{N-1}
\end{pmatrix}
$$

This matrix brings us closer to diagonalising $H$ because if we define $\mathbf{a}=(a_0, a_1, a_2, ..., a_{N-1})^\mathrm{T}$:

$$
H = \mathbf{a}^\dagger H^1 \mathbf{a}
$$

In [29]:
# Test tridiagonal structure
N = 5
J = np.linspace(1, 10, N-1)
h = 2 * np.ones(N)
H1 = np.empty((N, N))
zeroket = np.zeros((2**N))
zeroket[0] = 1
for n in range(N):
    for m in range(N):
        mat = (a(n) * Hamiltonian(N, J, h) * ad(m)).matrix()
        H1nm = zeroket @ mat @ zeroket
        H1nm = np.real_if_close(H1nm) 
        H1[n, m] = H1nm

print(f'J = {J}')
print(f'h = {h}')
print('H1 =')
print(H1)

J = [ 1.  4.  7. 10.]
h = [2. 2. 2. 2. 2.]
H1 =
[[ 2.  1.  0.  0.  0.]
 [ 1.  2.  4.  0.  0.]
 [ 0.  4.  2.  7.  0.]
 [ 0.  0.  7.  2. 10.]
 [ 0.  0.  0. 10.  2.]]


Given that the matrix $H^1$ is real symmetric, it is orthogonally diagonalisable:

$$
P^\mathrm{T}H^1P = \mathrm{diag}(\varepsilon_0, \varepsilon_1, ..., \varepsilon_{N-1}) 
$$

We can therefore define a new set of operators $\mathbf{\tilde{a}}=(\tilde{a}_0, \tilde{a}_1, \tilde{a}_2, ..., \tilde{a}_{N-1})^\mathrm{T}=P^\mathrm{T}\mathbf{a}$ in order to diagonalise $H$. We may check that these operators still satisfy the canonical anticommutation relations, and therefore are fermionic operators, on account of the fact that $\tilde{a}_n=\sum\limits_{j=0}^{N-1}P_{jn}a_j$ with $P$ an orthogonal matrix.

$$
\{\tilde{a}^\dagger_n, \tilde{a}_m\} = \sum\limits_{j=0}^{N-1}\sum\limits_{k=0}^{N-1}P_{jn}P_{km}\{a^\dagger_j, a_k\}=
\sum\limits_{j=0}^{N-1}\sum\limits_{k=0}^{N-1}P_{jn}P_{km}\delta_{jk}=\sum\limits_{j=0}^{N-1}P_{jn}P_{jm}=\delta_{nm}
$$

$$
\{\tilde{a}_n, \tilde{a}_m\} = \sum\limits_{j=0}^{N-1}\sum\limits_{k=0}^{N-1}P_{jn}P_{km}\{a_j, a_k\}=0
$$

Finally, the Hamiltonian is diagonal in the basis in which we excite fermions at sites $n_0, n_1, ..., n_k$, i.e.

$$
H = 
\sum\limits_{n=0}^{N-1}\varepsilon_n\tilde{a}_n^\dagger\tilde{a}_n = 
\sum\limits_{n=0}^{N-1}\varepsilon_n\tilde{N}_n 
$$

$$
H|n_0, n_1, ..., n_k\rangle = E(n_0, n_1, ..., n_k)|n_0, n_1, ..., n_k\rangle
$$

where

$$
\begin{aligned}
|n_0, n_1, ..., n_k\rangle &= \tilde{a}^\dagger_{n_0}\tilde{a}^\dagger_{n_1}...\tilde{a}^\dagger_{n_k}|0\rangle, &
E(n_0, n_1, ..., n_k) = \sum\limits_{j=0}^k \varepsilon_{n_j}
\end{aligned}
$$

In [30]:
# Putting into practice
N = 5
J = np.linspace(1, 10, N-1)
h = 2 * np.ones(N)
H1 = np.diag(h) + np.diag(J, 1) + np.diag(J, -1)
P = cirq.diagonalize_real_symmetric_matrix(H1)
E = np.diag(P.T @ H1 @ P)
print(f'Spectrum of H1: {E}')
# The ground state's Fermi energy is the largest negative one
print(f'Ground state energy: {np.sum(E[E<0])}')

Spectrum of H1: [-10.43759596  -1.36247032   2.           5.36247032  14.43759596]
Ground state energy: -11.80006628673773


In [31]:
# Do we obtain the same result via direct diagonalisation?
H = Hamiltonian(N, J, h).matrix()
E = np.linalg.eigh(H)[0]
print(f'Spectrum of H: {E}')
print(f'Ground state energy: {E[0]}')

Spectrum of H: [-11.80006629 -10.43759596  -9.80006629  -8.43759596  -6.43759596
  -5.07512564  -4.43759596  -3.07512564  -1.36247032   0.
   0.63752968   2.           2.63752968   4.           4.
   4.63752968   5.36247032   6.           6.           7.36247032
   8.           9.36247032  10.          11.36247032  13.07512564
  14.43759596  15.07512564  16.43759596  18.43759596  19.80006629
  20.43759596  21.80006629]
Ground state energy: -11.800066286737724


### References

[1] Latorre, J., & Riera, A. (2009). A short review on entanglement in quantum spin systems. *Journal of Physics A: Mathematical and Theoretical, 42(50), 504002*. Retrieved from https://arxiv.org/abs/0906.1499

[2] Finkel, F., & González-López, A. (2020). Inhomogeneous XX spin chains and quasi-exactly solvable models. *Journal of Statistical Mechanics: Theory and Experiment, 2020(9), 093105*. Retrieved from https://arxiv.org/abs/2007.00369

[3] Finkel, F., & González-López, A. (2021). Entanglement entropy of inhomogeneous XX spin chains with algebraic interactions. *Journal of High Energy Physics, 2021(12)*. Retrieved from https://arxiv.org/abs/2107.12200